# Step 1: Reference Data Analysis - Fresh OSM Download

**Download fresh street network data from OpenStreetMap**

**Cities** (500×500m windows):
- London, UK: 51.511°N, 0.130°W
- Berlin, Germany: 52.528°N, 13.403°E
- Belgrade, Serbia: 44.816°N, 20.462°E
- Torino, Italy: 45.069°N, 7.682°E

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Patch
from pathlib import Path
from collections import Counter
import math
import pickle
import osmnx as ox

# Configure OSMnx
ox.settings.log_console = True
ox.settings.use_cache = True

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded")
print(f"OSMnx version: {ox.__version__}")

## Configuration

In [ ]:
CITIES = {
    'london': {
        'name': 'London, UK',
        'coords': (51.5108708294874, -0.1301202729436442),
        'color': '#E74C3C'
    },
    'berlin': {
        'name': 'Berlin, Germany',
        'coords': (52.52832783083204, 13.40299924970717),
        'color': '#3498DB'
    },
    'belgrade': {
        'name': 'Belgrade, Serbia',
        'coords': (44.81648489551224, 20.462214816208164),
        'color': '#2ECC71'
    },
    'torino': {
        'name': 'Torino, Italy',
        'coords': (45.06940684010285, 7.682084193995683),
        'color': '#F39C12'
    }
}

WINDOW_SIZE_M = 500  # 500×500m window
MIN_SEGMENT_LENGTH = 5.0  # Filter segments < 5m

print(f"Window size: {WINDOW_SIZE_M}m × {WINDOW_SIZE_M}m")
print(f"Min segment length: {MIN_SEGMENT_LENGTH}m")

## Download and Process Networks

In [ ]:
def download_network(city_key, center_lat, center_lon, dist=250):
    """
    Download street network from OSM within radius of center point.
    
    Args:
        city_key: City identifier
        center_lat: Latitude of center
        center_lon: Longitude of center
        dist: Radius in meters (250m = 500m diameter)
    
    Returns:
        NetworkX graph (projected to UTM)
    """
    print(f"  Downloading from OSM (radius={dist}m)...")
    
    # Download walkable street network
    G = ox.graph_from_point(
        (center_lat, center_lon),
        dist=dist,
        network_type='walk',
        simplify=True
    )
    
    print(f"  Downloaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    # Project to UTM
    G_proj = ox.project_graph(G)
    
    # Simplify topology
    G_proj = ox.simplification.consolidate_intersections(
        G_proj,
        tolerance=10,
        rebuild_graph=True,
        dead_ends=False
    )
    
    print(f"  After simplification: {G_proj.number_of_nodes()} nodes, {G_proj.number_of_edges()} edges")
    
    return G_proj


def clean_and_filter_graph(G, min_length=5.0):
    """
    Filter short edges and clean graph.
    
    Args:
        G: NetworkX graph (projected)
        min_length: Minimum edge length in meters
    
    Returns:
        Cleaned graph, node positions dict
    """
    # Get node positions
    pos = {node: (data['x'], data['y']) for node, data in G.nodes(data=True)}
    
    # Filter edges by length
    edges_to_remove = []
    for u, v, key, data in G.edges(keys=True, data=True):
        length = data.get('length', 0)
        if length < min_length:
            edges_to_remove.append((u, v, key))
    
    G.remove_edges_from(edges_to_remove)
    print(f"  Removed {len(edges_to_remove)} edges < {min_length}m")
    
    # Remove isolated nodes
    isolated = list(nx.isolates(G))
    G.remove_nodes_from(isolated)
    for node in isolated:
        if node in pos:
            del pos[node]
    
    print(f"  Final: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    return G, pos


def normalize_to_window(pos, window_size=500):
    """
    Normalize coordinates to [0, window_size] box.
    
    Args:
        pos: Dict of {node: (x, y)}
        window_size: Target window size
    
    Returns:
        Normalized positions dict
    """
    if not pos:
        return {}
    
    coords = np.array(list(pos.values()))
    min_x, min_y = coords.min(axis=0)
    max_x, max_y = coords.max(axis=0)
    
    # Center and scale
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    
    pos_normalized = {}
    for node, (x, y) in pos.items():
        # Center at origin
        nx = x - center_x
        ny = y - center_y
        # Shift to positive quadrant
        nx += window_size / 2
        ny += window_size / 2
        pos_normalized[node] = (nx, ny)
    
    return pos_normalized


print("✓ Helper functions defined")

## Metrics Computation Functions

In [ ]:
def calculate_bearing(p1, p2):
    """Calculate bearing (0-180°)."""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = math.atan2(dy, dx)
    bearing = math.degrees(angle)
    if bearing < 0:
        bearing += 180
    if bearing >= 180:
        bearing -= 180
    return bearing


def compute_morphology_metrics(G, pos):
    """Compute morphology metrics."""
    metrics = {}
    
    area_km2 = (WINDOW_SIZE_M / 1000.0) ** 2
    metrics['node_density'] = G.number_of_nodes() / area_km2
    
    degrees = [d for _, d in G.degree()]
    metrics['degree_distribution'] = dict(Counter(degrees))
    metrics['avg_degree'] = np.mean(degrees) if degrees else 0
    
    dead_ends = sum(1 for d in degrees if d == 1)
    metrics['dead_end_ratio'] = dead_ends / len(degrees) if degrees else 0
    
    # Segment lengths
    lengths = []
    for u, v in G.edges():
        length = np.linalg.norm(np.array(pos[u]) - np.array(pos[v]))
        lengths.append(length)
    
    metrics['segment_lengths'] = lengths
    metrics['avg_segment_length'] = np.mean(lengths) if lengths else 0
    
    # Orientation
    bearings = []
    for u, v in G.edges():
        bearing = calculate_bearing(pos[u], pos[v])
        bearings.append(bearing)
    
    if bearings:
        counts, bins = np.histogram(bearings, bins=18, range=(0, 180))
        metrics['orientation_hist'] = (bins, counts)
    else:
        metrics['orientation_hist'] = (np.linspace(0, 180, 19), np.zeros(18))
    
    return metrics


def compute_space_syntax_metrics(G):
    """Compute space syntax metrics."""
    if G.number_of_nodes() < 2:
        return {
            'mean_depth': 0,
            'mean_depth_per_node': {},
            'local_integration': {},
            'choice': {},
            'intelligibility': 0
        }
    
    # Convert to undirected and use largest component
    G_undir = G.to_undirected()
    
    if not nx.is_connected(G_undir):
        largest_cc = max(nx.connected_components(G_undir), key=len)
        G_undir = G_undir.subgraph(largest_cc).copy()
    
    # Mean depth
    total_depth = 0
    count = 0
    for source in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, source)
        total_depth += sum(lengths.values())
        count += len(lengths)
    
    mean_depth = total_depth / count if count > 0 else 0
    
    # Mean depth per node
    mean_depth_per_node = {}
    for node in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, node)
        avg = np.mean(list(lengths.values())) if lengths else 0
        mean_depth_per_node[node] = avg
    
    # Local integration (R=3)
    local_int = {}
    for node in G_undir.nodes():
        lengths = nx.single_source_shortest_path_length(G_undir, node, cutoff=3)
        if len(lengths) > 1:
            total = sum(lengths.values())
            local_int[node] = (len(lengths) - 1) / total if total > 0 else 0
        else:
            local_int[node] = 0
    
    # Choice (betweenness)
    choice = nx.betweenness_centrality(G_undir, normalized=True)
    
    # Intelligibility
    degrees = [G_undir.degree(n) for n in local_int.keys()]
    integrations = list(local_int.values())
    
    if len(degrees) > 1 and np.std(degrees) > 0 and np.std(integrations) > 0:
        corr = np.corrcoef(degrees, integrations)[0, 1]
    else:
        corr = 0
    
    return {
        'mean_depth': mean_depth,
        'mean_depth_per_node': mean_depth_per_node,
        'local_integration': local_int,
        'choice': choice,
        'intelligibility': corr
    }


print("✓ Metrics functions defined")

## Download All Cities from OSM

In [ ]:
city_data = {}

print("Downloading fresh data from OpenStreetMap...\n")
print("="*70)

for city_key, city_info in CITIES.items():
    print(f"\n{city_info['name']}:")
    try:
        # Download
        lat, lon = city_info['coords']
        G = download_network(city_key, lat, lon, dist=250)
        
        # Clean
        G_clean, pos = clean_and_filter_graph(G, min_length=MIN_SEGMENT_LENGTH)
        
        # Normalize to window
        pos_norm = normalize_to_window(pos, WINDOW_SIZE_M)
        
        # Compute metrics
        morph = compute_morphology_metrics(G_clean, pos_norm)
        syntax = compute_space_syntax_metrics(G_clean)
        
        city_data[city_key] = {
            'graph': G_clean,
            'pos': pos_norm,
            'morphology': morph,
            'syntax': syntax
        }
        
        print(f"  ✓ Success!\n")
        
    except Exception as e:
        print(f"  ✗ Error: {e}\n")
        import traceback
        traceback.print_exc()

print("="*70)
print(f"\n✓ Loaded {len(city_data)} cities successfully\n")

## A1. Visualize All 4 Networks

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    color = CITIES[city_key]['color']
    
    # Window boundary
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='gray', linestyle='--', linewidth=1.5))
    
    # Draw edges
    for u, v in G.edges():
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        ax.plot(x, y, color=color, linewidth=1.5, alpha=0.7, zorder=1)
    
    # Draw nodes colored by degree
    degrees = dict(G.degree())
    max_degree = max(degrees.values()) if degrees else 1
    
    for node in G.nodes():
        degree = degrees[node]
        color_val = degree / max_degree
        node_color = plt.cm.RdYlBu_r(color_val)
        
        ax.scatter(pos[node][0], pos[node][1], s=40, c=[node_color],
                  zorder=2, edgecolors='black', linewidths=0.5)
    
    ax.set_xlim(-20, WINDOW_SIZE_M + 20)
    ax.set_ylim(-20, WINDOW_SIZE_M + 20)
    ax.set_aspect('equal')
    ax.set_title(
        f"{CITIES[city_key]['name']}\n{G.number_of_nodes()} nodes, {G.number_of_edges()} edges",
        fontsize=12, fontweight='bold'
    )
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Reference Street Networks (nodes colored by degree)', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A2. Summary Statistics

In [ ]:
summary_rows = []

for city_key, data in city_data.items():
    G = data['graph']
    morph = data['morphology']
    syntax = data['syntax']
    
    summary_rows.append({
        'City': CITIES[city_key]['name'],
        'Nodes': G.number_of_nodes(),
        'Edges': G.number_of_edges(),
        'Density (n/km²)': f"{morph['node_density']:.1f}",
        'Avg Degree': f"{morph['avg_degree']:.2f}",
        'Dead-End Ratio': f"{morph['dead_end_ratio']:.3f}",
        'Avg Seg Length (m)': f"{morph['avg_segment_length']:.1f}",
        'Mean Depth': f"{syntax['mean_depth']:.2f}",
        'Intelligibility': f"{syntax['intelligibility']:.3f}"
    })

df_summary = pd.DataFrame(summary_rows)
print("\n" + "="*100)
print(" "*35 + "REFERENCE CITIES SUMMARY")
print("="*100)
print(df_summary.to_string(index=False))
print("="*100)

## A3. Degree Distribution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    degree_dist = data['morphology']['degree_distribution']
    
    degrees = sorted(degree_dist.keys())
    counts = [degree_dist[d] for d in degrees]
    total = sum(counts)
    probs = [c / total for c in counts]
    
    color = CITIES[city_key]['color']
    ax.bar(degrees, probs, color=color, alpha=0.7, edgecolor='black', linewidth=1)
    ax.set_xlabel('Node Degree', fontsize=11)
    ax.set_ylabel('Probability', fontsize=11)
    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=12)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xticks(degrees)

plt.suptitle('Degree Distributions', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## A4. Orientation Rose Diagrams

In [ ]:
fig = plt.figure(figsize=(14, 10))

for idx, (city_key, data) in enumerate(city_data.items(), 1):
    ax = fig.add_subplot(2, 2, idx, projection='polar')
    
    bins, counts = data['morphology']['orientation_hist']
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Convert to radians and double for full circle
    theta = np.deg2rad(bin_centers)
    theta_full = np.concatenate([theta, theta + np.pi])
    counts_full = np.concatenate([counts, counts])
    
    # Normalize
    total = sum(counts_full)
    probs = counts_full / total if total > 0 else counts_full
    
    color = CITIES[city_key]['color']
    width = np.deg2rad(bins[1] - bins[0])
    
    ax.bar(theta_full, probs, width=width, color=color, alpha=0.7, 
           edgecolor='black', linewidth=0.5)
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=12, pad=20)
    ax.set_ylim(0, max(probs) * 1.1 if max(probs) > 0 else 0.1)

plt.suptitle('Street Orientation Rose Diagrams', fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

## A5. Local Integration Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    local_int = data['syntax']['local_integration']
    
    # Window
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))
    
    # Draw edges (gray)
    for u, v in G.edges():
        x = [pos[u][0], pos[v][0]]
        y = [pos[u][1], pos[v][1]]
        ax.plot(x, y, color='lightgray', linewidth=1, alpha=0.5, zorder=1)
    
    # Draw nodes colored by local integration
    values = list(local_int.values())
    if values:
        vmin, vmax = min(values), max(values)
        
        for node in G.nodes():
            if node in local_int:
                val = local_int[node]
                norm_val = (val - vmin) / (vmax - vmin + 1e-10)
                color = plt.cm.hot(norm_val)
                
                ax.scatter(pos[node][0], pos[node][1], s=60, c=[color],
                          zorder=2, edgecolors='black', linewidths=0.5)
    
    ax.set_xlim(-20, WINDOW_SIZE_M + 20)
    ax.set_ylim(-20, WINDOW_SIZE_M + 20)
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nLocal Integration (R=3)",
                fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Local Integration Maps (warm = high integration)',
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A6. Choice (Betweenness) Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    choice = data['syntax']['choice']
    
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))
    
    # Edges
    for u, v in G.edges():
        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]],
               color='lightgray', linewidth=1, alpha=0.5, zorder=1)
    
    # Nodes colored by choice
    values = list(choice.values())
    if values:
        vmin, vmax = min(values), max(values)
        
        for node in G.nodes():
            if node in choice:
                val = choice[node]
                norm_val = (val - vmin) / (vmax - vmin + 1e-10)
                color_val = plt.cm.viridis(norm_val)
                
                ax.scatter(pos[node][0], pos[node][1], s=60, c=[color_val],
                          zorder=2, edgecolors='black', linewidths=0.5)
    
    ax.set_xlim(-20, WINDOW_SIZE_M + 20)
    ax.set_ylim(-20, WINDOW_SIZE_M + 20)
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nChoice (Betweenness)",
                fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Choice Maps (through-movement corridors)',
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A7. Mean Depth Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    mean_depth_nodes = data['syntax']['mean_depth_per_node']
    
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))
    
    # Edges
    for u, v in G.edges():
        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]],
               color='lightgray', linewidth=1, alpha=0.5, zorder=1)
    
    # Nodes colored by mean depth
    values = list(mean_depth_nodes.values())
    if values:
        vmin, vmax = min(values), max(values)
        
        for node in G.nodes():
            if node in mean_depth_nodes:
                val = mean_depth_nodes[node]
                norm_val = (val - vmin) / (vmax - vmin + 1e-10)
                color_val = plt.cm.coolwarm_r(norm_val)
                
                ax.scatter(pos[node][0], pos[node][1], s=60, c=[color_val],
                          zorder=2, edgecolors='black', linewidths=0.5)
    
    ax.set_xlim(-20, WINDOW_SIZE_M + 20)
    ax.set_ylim(-20, WINDOW_SIZE_M + 20)
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nMean Depth",
                fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Mean Depth Maps (blue = central, red = peripheral)',
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A8. Intelligibility Scatter

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph'].to_undirected()
    local_int = data['syntax']['local_integration']
    intelligibility = data['syntax']['intelligibility']
    
    degrees = [G.degree(n) for n in local_int.keys()]
    integrations = list(local_int.values())
    
    color = CITIES[city_key]['color']
    ax.scatter(degrees, integrations, alpha=0.6, s=30, c=color,
              edgecolors='black', linewidths=0.5)
    
    # Fit line
    if len(degrees) > 1:
        z = np.polyfit(degrees, integrations, 1)
        p = np.poly1d(z)
        x_line = np.linspace(min(degrees), max(degrees), 100)
        ax.plot(x_line, p(x_line), 'r--', linewidth=2, alpha=0.7)
    
    ax.set_xlabel('Connectivity (Degree)', fontsize=11)
    ax.set_ylabel('Local Integration', fontsize=11)
    ax.set_title(f"{CITIES[city_key]['name']}\nr = {intelligibility:.3f}",
                fontweight='bold', fontsize=12)
    ax.grid(True, alpha=0.3)

plt.suptitle('Intelligibility: Degree vs Local Integration',
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## A9. Cross-City Segment Lengths

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    lengths = data['morphology']['segment_lengths']
    
    if lengths:
        counts, bins = np.histogram(lengths, bins=20, range=(MIN_SEGMENT_LENGTH, 120))
        bin_centers = (bins[:-1] + bins[1:]) / 2
        total = sum(counts)
        probs = counts / total if total > 0 else counts
        
        color = CITIES[city_key]['color']
        ax.bar(bin_centers, probs, width=(bins[1]-bins[0])*0.9,
              color=color, alpha=0.7, edgecolor='black', linewidth=0.5)
    
    ax.set_xlabel('Length (m)', fontsize=10)
    ax.set_ylabel('Probability' if idx == 0 else '', fontsize=10)
    ax.set_title(CITIES[city_key]['name'], fontweight='bold', fontsize=11)
    ax.set_ylim(0, 0.2)
    ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Segment Length Distributions (Aligned)',
            fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## A13. Node Degree Maps

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

degree_colors = {1: 'red', 2: 'orange', 3: 'yellow', 4: 'green', 5: 'blue', 6: 'purple'}

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))
    
    # Edges
    for u, v in G.edges():
        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]],
               color='lightgray', linewidth=1, alpha=0.4, zorder=1)
    
    # Nodes sized and colored by degree
    degrees = dict(G.degree())
    
    for node in G.nodes():
        degree = degrees[node]
        size = 30 + degree * 20
        color = degree_colors.get(degree, 'gray')
        
        ax.scatter(pos[node][0], pos[node][1], s=size, c=color,
                  zorder=2, edgecolors='black', linewidths=0.5, alpha=0.8)
    
    ax.set_xlim(-20, WINDOW_SIZE_M + 20)
    ax.set_ylim(-20, WINDOW_SIZE_M + 20)
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nNode Degree Map",
                fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

plt.suptitle('Node Degree Maps (size & color by degree)',
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## A14. Core + Corridor Overlap

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
axes = axes.flatten()

TOP_PERCENT = 0.2

for idx, (city_key, data) in enumerate(city_data.items()):
    ax = axes[idx]
    G = data['graph']
    pos = data['pos']
    local_int = data['syntax']['local_integration']
    choice = data['syntax']['choice']
    
    ax.add_patch(Rectangle((0, 0), WINDOW_SIZE_M, WINDOW_SIZE_M,
                           fill=False, edgecolor='gray', linestyle='--', linewidth=1))
    
    # Find top 20%
    int_values = sorted(local_int.values(), reverse=True)
    choice_values = sorted(choice.values(), reverse=True)
    
    int_threshold = int_values[int(len(int_values) * TOP_PERCENT)] if int_values else 0
    choice_threshold = choice_values[int(len(choice_values) * TOP_PERCENT)] if choice_values else 0
    
    high_int = {n for n, v in local_int.items() if v >= int_threshold}
    high_choice = {n for n, v in choice.items() if v >= choice_threshold}
    overlap = high_int & high_choice
    
    # Edges
    for u, v in G.edges():
        ax.plot([pos[u][0], pos[v][0]], [pos[u][1], pos[v][1]],
               color='lightgray', linewidth=1, alpha=0.3, zorder=1)
    
    # All nodes (small gray)
    for node in G.nodes():
        ax.scatter(pos[node][0], pos[node][1], s=10, c='lightgray',
                  zorder=2, alpha=0.5)
    
    # High integration only (blue)
    for node in high_int - overlap:
        ax.scatter(pos[node][0], pos[node][1], s=60, c='blue',
                  zorder=3, edgecolors='black', linewidths=0.5, alpha=0.7)
    
    # High choice only (green)
    for node in high_choice - overlap:
        ax.scatter(pos[node][0], pos[node][1], s=60, c='green',
                  zorder=3, edgecolors='black', linewidths=0.5, alpha=0.7)
    
    # Overlap (red)
    for node in overlap:
        ax.scatter(pos[node][0], pos[node][1], s=80, c='red',
                  zorder=4, edgecolors='black', linewidths=1, alpha=0.9)
    
    ax.set_xlim(-20, WINDOW_SIZE_M + 20)
    ax.set_ylim(-20, WINDOW_SIZE_M + 20)
    ax.set_aspect('equal')
    ax.set_title(f"{CITIES[city_key]['name']}\nCore+Corridor ({len(overlap)} overlap)",
                fontweight='bold', fontsize=12)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.grid(True, alpha=0.3)

legend_elements = [
    Patch(facecolor='blue', edgecolor='black', label='High Integration'),
    Patch(facecolor='green', edgecolor='black', label='High Choice'),
    Patch(facecolor='red', edgecolor='black', label='Overlap (Both)')
]
fig.legend(handles=legend_elements, loc='upper center', ncol=3,
          bbox_to_anchor=(0.5, 0.99))

plt.suptitle('Core + Corridor Overlap (Top 20%)',
            fontsize=16, fontweight='bold', y=0.975)
plt.tight_layout()
plt.show()

## Save Reference Data

In [ ]:
with open('reference_cities_data.pkl', 'wb') as f:
    pickle.dump(city_data, f)

print("✓ Reference data saved to: reference_cities_data.pkl")
print(f"\nCities saved: {list(city_data.keys())}")
print("\nReady for Step 2: Network Generation")